# BatFlightVisualization3D-compare

## Paquetes

In [35]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from matplotlib import cm, colors

from BatFlightAnalysis import read_data, split_data, animate_flower_bat

## Código

In [48]:
def plot_flower_bat(individuo, whiskers, pedicelo):
    # Crear el gráfico 3D interactivo fuera del bucle
    fig = go.Figure()

    # Generar una paleta de 20 colores usando una escala continua (ej. Viridis)
    num_trayectorias = 20
    colormap = cm.get_cmap("viridis", num_trayectorias)
    colores = [colors.rgb2hex(colormap(i)) for i in range(colormap.N)]

    # Punto de referencia para iniciar todas las trayectorias
    punto_referencia = [0, 0, 0]

    # Inicializar valores mínimos y máximos para los ejes
    x_min, x_max = float('inf'), float('-inf')
    y_min, y_max = float('inf'), float('-inf')
    z_min, z_max = float('inf'), float('-inf')

    # Añadir el punto de inicio
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers', marker=dict(size=5, color='red'),
        name=f'Flor'
    ))

    # Inicio 1 si pedicelo Flexible, 2 si pedicelo Estable
    inicio = {'Flexible': 2, 'Estable': 1}[pedicelo]

    for i in range(inicio, num_trayectorias + 1, 2):  # Cambia el rango según la cantidad de trayectorias
        # print(f'Procesando vuelo {i}')
        data_bef, data_fee, data_aft = read_data(f'data/Individuo {individuo}/{whiskers}/output_points_{i}.xlsx')
        flower_bef, flower_fee, flower_aft, bat_bef, bat_fee, bat_aft = split_data(data_bef, data_fee, data_aft)
        df = flower_fee.copy()

        # Transformar la trayectoria para que inicie en el punto de referencia
        offset = [df['x'].iloc[0], df['y'].iloc[0], df['z'].iloc[0]]
        df['x'] -= offset[0]
        df['y'] -= offset[1]
        df['z'] -= offset[2]

        # Actualizar valores mínimos y máximos
        x_min, x_max = min(x_min, df['x'].min()), max(x_max, df['x'].max())
        y_min, y_max = min(y_min, df['y'].min()), max(y_max, df['y'].max())
        z_min, z_max = min(z_min, df['z'].min()), max(z_max, df['z'].max())

        # Asignar un color diferente basado en la paleta generada
        color = colores[i - 1]

        # Añadir la trayectoria con todos los puntos menos el último en línea entrecortada
        fig.add_trace(go.Scatter3d(
            x=df['x'][:-1], y=df['y'][:-1], z=df['z'][:-1],
            mode='lines', line=dict(width=2, dash='dash', color=color),
            name=f'Vuelo {i}'
        ))

    # Configurar el diseño del gráfico usando los valores min y max
    fig.update_layout(
        scene=dict(
            xaxis=dict(showticklabels=False, tickmode="linear", dtick=10, range=[x_min - 5, x_max + 5], backgroundcolor="lightblue", gridwidth=0.5),
            yaxis=dict(showticklabels=False, tickmode="linear", dtick=10, range=[y_min - 5, y_max + 5], backgroundcolor="lightblue", gridwidth=0.5),
            zaxis=dict(showticklabels=False, tickmode="linear", dtick=10, range=[z_min - 5, z_max + 5], backgroundcolor="lightblue", gridwidth=0.5)
        ),
        title=f'Trayectoria {whiskers} - {pedicelo} - Individuo {individuo}',
    )

    # Configurar la cámara para que salga cuadrado y alejado de los puntos
    camera = dict(
        eye=dict(x=1.5, y=1.5, z=1.5)
    )
    fig.update_layout(scene_camera=camera) 
    fig.show()

In [49]:
plot_flower_bat(7, 'Whiskers', 'Flexible')
plot_flower_bat(7, 'No whiskers', 'Flexible')
plot_flower_bat(7, 'Whiskers', 'Estable')
plot_flower_bat(7, 'No whiskers', 'Estable')